Here we use the RTS 2 Annex 3 model to generate test trade data which we then classify.

In these first examples we generate a couple of really simple cases:
* Generate a sample trade for a specific Sub-asset class
* Generate a sample trade for a specific Asset class and a random Sub-asset class

In [6]:
import rts2_annex3
import json

root = rts2_annex3.class_root

# Generate a test sample for a sub-asset class with no criteria
sub_asset_class = root.sub_asset_class_by_name(
    sub_asset_class_name="Exchange Traded Notes (ETNs)")
sample = sub_asset_class.make_test_samples(1)[0]
classification = root.classification_for(subject=sample)
print(json.dumps(classification.classification_dict(), indent=4))

# Now make a sample for a specifc asset class, but let the sub-asset class be random
asset_class = root.asset_class_by_name(asset_class_name="Bonds (ETC and ETN bond types)")
sample = asset_class.make_test_samples(1)[0]
classification = root.classification_for(subject=sample)
print(json.dumps(classification.classification_dict(), indent=4))


Generating samples for Exchange Traded Notes (ETNs).
{
    "RTS2 version": "EU 2017/583 of 14 July 2016",
    "Asset class": "Bonds (ETC and ETN bond types)",
    "Sub-asset class": "Exchange Traded Notes (ETNs)"
}
Generating samples for Bonds (ETC and ETN bond types).
Generating samples for Exchange Traded Commodities (ETCs).
{
    "RTS2 version": "EU 2017/583 of 14 July 2016",
    "Asset class": "Bonds (ETC and ETN bond types)",
    "Sub-asset class": "Exchange Traded Commodities (ETCs)"
}
